In [1]:
using NBInclude
@nbinclude("datastructs.ipynb")
@nbinclude("helpers.ipynb")

update_adj_set (generic function with 2 methods)

In [2]:
# given ge, finds next ge (with new gene) in the next adj

# e.g., given adjs={ge1:h, ge2:t} ... {ge2:h, ge3:t} & ge=ge2:t, 
# finds {ge2:h, ge3:t}, returns ge3:t 
function find_next_adj(ge::GeneEnd, adj::Adjacency, reordered_adj_list::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adjs::Vector{Adjacency}, processed::Set{Int}) 
    (ge.head == true) ? opp_th_idx = 1 : opp_th_idx = 2  

    # idx of {ge2:h, ge3:t}
    next_idx = src_gid_to_l[ge.gene.id][opp_th_idx]
    next_adj = src_adjs[next_idx]

    if next_idx ∉ processed 
        push!(processed, next_idx)
        push!(reordered_adj_list, next_adj)
    end 
    
    if next_adj.left.gene == ge.gene
        ge = other_adjacency_end(next_adj.left, next_adj)
    elseif next_adj.right.gene == ge.gene
        ge = other_adjacency_end(next_adj.right, next_adj)
    end 
    
    return ge 
end 

# prints out adj list in order s.t. neighboring adj have h/t gene end pairs 
# O(N), n = length source adjacencies 
# can reconstruct genome from this in O(N)
# (note that order within the adj don't matter)
function reorder_adjs(src_adjs::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}})
    telos = Vector{Int}() 
    circ = Vector{Int}() 
    processed = Set{Int}()
    reordered_adj_list = Vector{Adjacency}()

    for i in 1:length(src_adjs)
        adj = src_adjs[i]
        if adj.left.gene == Telomere() || adj.right.gene == Telomere() 
            push!(telos, i)
        else 
            push!(circ, i)
        end 
    end 
    
    # handle telomeres
    for i in telos 
        if i in processed 
            continue 
        else 
            push!(processed, i)
        end
        
        adj = src_adjs[i]
        push!(reordered_adj_list, adj)

        ge = other_adjacency_end(Telomere(), adj)
        while ge.gene != Telomere()  
            ge = find_next_adj(ge, adj, reordered_adj_list, src_gid_to_l, src_adjs, processed)
        end 
    end 
    
    # handle adj with two genes
    for i in circ  
        if i in processed 
            continue 
        else 
            push!(processed, i)
        end

        adj = src_adjs[i]
        push!(reordered_adj_list, adj)

        first_gene = adj.left.gene 
        ge = adj.right 
        first = true       
        
        while ge.gene != first_gene || first == true  
            ge = find_next_adj(ge, adj, reordered_adj_list, src_gid_to_l, src_adjs, processed)

            first = false 
        end   
    end 
    return reordered_adj_list
end 

reorder_adjs (generic function with 1 method)